In [5]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
import numpy as np
import pandas as pd
from merf.merf import MERF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce

In [9]:
path = "/Users/gauthier.gadouas/Desktop/Projet AFM/all_data.xlsx"
df = pd.ExcelFile(path).parse()

#binary encoding
df["Sexe"] = df["Sexe"].apply(lambda x: 1 if x == "F" else 0)
df["Primitif/ méta/\n"] = df["Primitif/ méta/\n"].apply(lambda x: 1 if x == "Primitif" else 0)
df["Localisation"] = df["Localisation"].apply(lambda x: 1 if x == "Colon gauche" else 0)
df["Composante mucineuse"] = df["Composante mucineuse"].apply(lambda x: 1 if x == "Oui" else 0)
df["Emboles \n"] = df["Emboles \n"].apply(lambda x: 1 if x == "Oui" else 0)
df["Infiltration perinerveuse\n"] = df["Infiltration perinerveuse\n"].apply(lambda x: 1 if x == "Oui" else 0)
df["Microsatellite"] = df["Microsatellite"].apply(lambda x: 1 if x == "MSI" else 0)
df["BRAF"] = df["BRAF"].apply(lambda x: 0 if x == "NM" else 1)
df["RAS+BRAF"] = df["RAS+BRAF"].apply(lambda x: 1 if x == "Muté" else 0)
df["Stade"] = df["Stade"].apply(lambda x: 1 if x == "Stade III" or x == "Stade IV" else 0)
df["RAS"] = df["RAS"].apply(lambda x: 0 if x == "NM" else 1)
df["KRAS exon 2"] = df["KRAS exon 2"].apply(lambda x: 0 if x == "NM" else 1)
df["NRAS exon2"] = df["NRAS exon2"].apply(lambda x: 0 if x == "NM" else 1)

#See weight of evidence / Helmert encoding for categorical encoding

#DDN : min DDN and days since DDN. Same for DDD
min_timestamp_DDN = df['DDN'].min()
min_timestamp_DDD = df['Date de diagnostic'].min()
# Calculate the difference in days and replace the original timestamp column
df['DDN'] = ((df['DDN'] - min_timestamp_DDN).dt.days) / 365.25
df['Date de diagnostic'] = ((df['Date de diagnostic'] - min_timestamp_DDD).dt.days) / 365.25

#Reste à gérer Tissu/Map/mutations

encoder = ce.HelmertEncoder(cols=['Tissu', 'Map'])
df = encoder.fit_transform(df)

/Users/gauthier.gadouas/.conda/envs/AFM-MERF/lib/python3.8/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
/Users/gauthier.gadouas/.conda/envs/AFM-MERF/lib/python3.8/site-packages/category_encoders/base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


,intercept,Data,Log Data,Tissu_0,Tissu_1,Tissu_2,Tissu_3,Tissu_4,Tissu_5,Tissu_6,...,Composante mucineuse,Emboles \n,Infiltration perinerveuse\n,Stade,KRAS exon 2,NRAS exon2,RAS,BRAF,RAS+BRAF,Microsatellite
0,1,833.6780,2.921519,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,1,0,0,0,0,0,0
1,1,1137.7700,3.056436,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,1,0,0,0,0,0,0
2,1,864.4180,2.937226,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,1,0,0,0,0,0,0
3,1,1321.6900,3.121458,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,1,0,0,0,0,0,0
4,1,1945.6100,3.289279,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7729,1,64.8878,1.818805,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,0,0,0,0,0,0,0
7730,1,139.3290,2.147147,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,0,0,0,0,0,0,0
7731,1,138.0480,2.143165,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,0,0,0,0,0,0,0
7732,1,117.6420,2.074238,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,1,1,0,0,0,0,0,0,0


In [12]:
X = df.drop(['Data', 'Log Data'], axis=1)
Y = df["Data"]
X_train, X_test, Y_train, Y_test = train_test_split(
X, Y, test_size=0.33, random_state=42, shuffle=True)
#Z_train = X_train[["Map"]]
#Z_test = X_test[["Map"]]
#clusters_train = X_train["Numéro Patient AFM"]
#clusters_test = X_test["Numéro Patient AFM"]
#X_train = X_train.drop(['Numéro Patient AFM', 'Map'], axis=1)
#X_test = X_test.drop(['Numéro Patient AFM', 'Map'], axis=1)

In [13]:
h = X_train.groupby("Numéro Patient AFM").count()#["Numéro Patient AFM"].hist()
h

,intercept,Tissu_0,Tissu_1,Tissu_2,Tissu_3,Tissu_4,Tissu_5,Tissu_6,Tissu_7,Tissu_8,...,Composante mucineuse,Emboles \n,Infiltration perinerveuse\n,Stade,KRAS exon 2,NRAS exon2,RAS,BRAF,RAS+BRAF,Microsatellite
Numéro Patient AFM,,,,,,,,,,,,,,,,,,,,,
1,417,417,417,417,417,417,417,417,417,417,...,417,417,417,417,417,417,417,417,417,417
2,238,238,238,238,238,238,238,238,238,238,...,238,238,238,238,238,238,238,238,238,238
3,235,235,235,235,235,235,235,235,235,235,...,235,235,235,235,235,235,235,235,235,235
4,465,465,465,465,465,465,465,465,465,465,...,465,465,465,465,465,465,465,465,465,465
5,627,627,627,627,627,627,627,627,627,627,...,627,627,627,627,627,627,627,627,627,627
6,341,341,341,341,341,341,341,341,341,341,...,341,341,341,341,341,341,341,341,341,341
7,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
8,172,172,172,172,172,172,172,172,172,172,...,172,172,172,172,172,172,172,172,172,172
9,167,167,167,167,167,167,167,167,167,167,...,167,167,167,167,167,167,167,167,167,167


#### We start fitting a MERF model

For MERF, we need to separate fixed effects from random effects
Let's assume clinical parameters are our fixed effects and 'Patient'/'Map' are our random effects

Note to myself: Careful with indices

In [82]:
fixed_effects_columns = [col for col in X_train.columns if col != "Numéro Patient AFM"]# and "Map_" not in col] # Assuming 'Treatment_A' as baseline, adjust as needed

X_fixed_train = X_train[fixed_effects_columns]
X_fixed_test = X_test[fixed_effects_columns]

random_effects_columns = [col for col in X_train.columns if 'Numéro Patient AFM' in col or 'Map_' in col]
clusters_train = X_train[random_effects_columns]
clusters_test = X_test[random_effects_columns]

Z_train = np.ones(shape=(X_fixed_train.shape[0],clusters_train.shape[1]))
Z_test = np.ones(shape=(X_fixed_test.shape[0], clusters_train.shape[1]))

#random_effects_columns = [col for col in X_train.columns if 'Numéro Patient AFM' in col or 'Map_' in col]
#Z_train = X_train[random_effects_columns]
#Z_test = X_test[random_effects_columns]

In [84]:
Z_train.shape

(5181, 114)

In [85]:
X_fixed_train = X_fixed_train.reset_index(drop=True)
clusters_train = clusters_train.reset_index(drop=True)
#Z_train = Z_train.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)

In [86]:
# Initializing and fitting the MERF model
merf = MERF()
merf.fit(X_fixed_train, Z_train, clusters_train, Y_train)#, pd.Series([1]*len(Y_train)))
y_pred = merf.predict(X_fixed_test, Z_test, clusters_test)
y_pred

TypeError: clusters must be a pandas Series.

In [74]:
(Y_test.to_numpy() - y_pred).mean()

316.08139551500096

In [80]:
Z_train.shape

(5181, 1)

In [14]:
# Random Forest Training and Prediction
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, Y_train)
yhat_rf = rf.predict(X_test)

We plot the error

In [17]:
# Compute MSE
mse_rf = np.sqrt(np.sum((Y - yhat_rf)**2)) / len(Y)
mse_mrf = np.sqrt(np.sum((Y - yhat_mrf)**2)) / len(Y)

# Plot Scatter Plot of error
plt.plot(Y, yhat_rf, 'o', label='RF')
plt.plot(Y, yhat_mrf, 'o', label='MERF')
plt.grid('on')
plt.xlabel('y')
plt.ylabel('y_hat')

plt.legend()
plt.title('MSE RF = {:.3f}, MSE MERF = {:.3f}'.format(mse_rf, mse_mrf))

ValueError: operands could not be broadcast together with shapes (7734,) (2553,) 

In [81]:
min(df.DDN)

0